# توجہ کے میکانزم اور ٹرانسفارمرز

ریکرنٹ نیٹ ورکس کا ایک بڑا نقص یہ ہے کہ کسی بھی ترتیب میں تمام الفاظ کا نتیجے پر یکساں اثر ہوتا ہے۔ اس کی وجہ سے معیاری LSTM انکوڈر-ڈیکوڈر ماڈلز میں ترتیب سے ترتیب کے کاموں، جیسے کہ نامزد ادارہ کی شناخت اور مشینی ترجمہ، میں کارکردگی کمزور ہو جاتی ہے۔ حقیقت میں، ان پٹ ترتیب کے مخصوص الفاظ اکثر دیگر الفاظ کے مقابلے میں ترتیب وار نتائج پر زیادہ اثر ڈالتے ہیں۔

سوچیے کہ ایک ترتیب سے ترتیب ماڈل، جیسے کہ مشینی ترجمہ۔ یہ دو ریکرنٹ نیٹ ورکس کے ذریعے نافذ کیا جاتا ہے، جہاں ایک نیٹ ورک (**انکوڈر**) ان پٹ ترتیب کو ایک چھپی ہوئی حالت میں تبدیل کرتا ہے، اور دوسرا نیٹ ورک، **ڈیکوڈر**، اس چھپی ہوئی حالت کو ترجمہ شدہ نتیجے میں کھولتا ہے۔ اس طریقے کے ساتھ مسئلہ یہ ہے کہ نیٹ ورک کی آخری حالت کو جملے کے آغاز کو یاد رکھنے میں مشکل پیش آتی ہے، جس کی وجہ سے لمبے جملوں پر ماڈل کی کارکردگی خراب ہو جاتی ہے۔

**توجہ کے میکانزم** ہر ان پٹ ویکٹر کے سیاق و سباق کے اثر کو ہر آؤٹ پٹ پیش گوئی پر وزن دینے کا ایک ذریعہ فراہم کرتے ہیں۔ اس کو نافذ کرنے کا طریقہ یہ ہے کہ ان پٹ RNN کی درمیانی حالتوں اور آؤٹ پٹ RNN کے درمیان شارٹ کٹس بنائے جائیں۔ اس طرح، جب آؤٹ پٹ علامت $y_t$ پیدا کی جا رہی ہو، تو ہم تمام ان پٹ چھپی ہوئی حالتوں $h_i$ کو مختلف وزن کے گتانک $\alpha_{t,i}$ کے ساتھ مدنظر رکھیں گے۔

![ایک انکوڈر/ڈیکوڈر ماڈل کی تصویر جس میں ایک اضافی توجہ کی تہہ شامل ہے](../../../../../translated_images/ur/encoder-decoder-attention.7a726296894fb567.webp)
*انکوڈر-ڈیکوڈر ماڈل اضافی توجہ کے میکانزم کے ساتھ [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) میں، [اس بلاگ پوسٹ](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html) سے لیا گیا*

توجہ میٹرکس $\{\alpha_{i,j}\}$ اس حد کو ظاہر کرے گی کہ کس حد تک مخصوص ان پٹ الفاظ آؤٹ پٹ ترتیب میں دیے گئے لفظ کی تخلیق میں کردار ادا کرتے ہیں۔ نیچے ایک ایسی میٹرکس کی مثال دی گئی ہے:

![RNNsearch-50 کے ذریعے پائی گئی ایک نمونہ ترتیب کی تصویر، Bahdanau - arviz.org سے لی گئی](../../../../../translated_images/ur/bahdanau-fig3.09ba2d37f202a6af.webp)

*[Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (شکل 3) سے لی گئی تصویر*

توجہ کے میکانزم موجودہ یا قریب موجودہ قدرتی زبان کی پروسیسنگ میں آرٹ کی حالت کے لیے بڑی حد تک ذمہ دار ہیں۔ تاہم، توجہ شامل کرنے سے ماڈل کے پیرامیٹرز کی تعداد میں بہت زیادہ اضافہ ہوتا ہے، جس کی وجہ سے RNNs کے ساتھ اسکیلنگ کے مسائل پیدا ہوتے ہیں۔ RNNs کو اسکیل کرنے کی ایک اہم رکاوٹ یہ ہے کہ ماڈلز کی ریکرنٹ نوعیت تربیت کو بیچ اور متوازی بنانے میں چیلنج پیدا کرتی ہے۔ RNN میں ترتیب کے ہر عنصر کو ترتیب وار عمل کرنا ضروری ہوتا ہے، جس کا مطلب ہے کہ اسے آسانی سے متوازی نہیں بنایا جا سکتا۔

توجہ کے میکانزم کو اپنانے اور اس رکاوٹ کے امتزاج نے ان ٹرانسفارمر ماڈلز کی تخلیق کی راہ ہموار کی جو آج ہم BERT سے لے کر OpenGPT3 تک استعمال کرتے ہیں۔

## ٹرانسفارمر ماڈلز

ہر پچھلی پیش گوئی کے سیاق و سباق کو اگلے جائزہ مرحلے میں آگے بڑھانے کے بجائے، **ٹرانسفارمر ماڈلز** **پوزیشنل انکوڈنگز** اور توجہ کا استعمال کرتے ہیں تاکہ دیے گئے ان پٹ کے سیاق و سباق کو فراہم کردہ متن کی ونڈو کے اندر قید کیا جا سکے۔ نیچے دی گئی تصویر دکھاتی ہے کہ کس طرح پوزیشنل انکوڈنگز کے ساتھ توجہ دی گئی ونڈو کے اندر سیاق و سباق کو قید کر سکتی ہے۔

![ایک متحرک GIF جو دکھاتا ہے کہ ٹرانسفارمر ماڈلز میں جائزے کیسے کیے جاتے ہیں۔](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

چونکہ ہر ان پٹ پوزیشن کو آزادانہ طور پر ہر آؤٹ پٹ پوزیشن پر نقشہ بنایا جاتا ہے، ٹرانسفارمرز RNNs کے مقابلے میں بہتر متوازی بن سکتے ہیں، جو کہ بہت بڑے اور زیادہ اظہار کرنے والے زبان کے ماڈلز کو قابل بناتا ہے۔ ہر توجہ کا سر مختلف الفاظ کے درمیان تعلقات سیکھنے کے لیے استعمال کیا جا سکتا ہے، جو نیچے کی قدرتی زبان کی پروسیسنگ کے کاموں کو بہتر بناتا ہے۔

**BERT** (Bidirectional Encoder Representations from Transformers) ایک بہت بڑا ملٹی لیئر ٹرانسفارمر نیٹ ورک ہے جس میں *BERT-base* کے لیے 12 تہیں، اور *BERT-large* کے لیے 24 تہیں ہیں۔ ماڈل کو پہلے بڑے متن کے ڈیٹا کارپس (ویکیپیڈیا + کتابیں) پر غیر نگرانی شدہ تربیت (ایک جملے میں ماسک کیے گئے الفاظ کی پیش گوئی) کا استعمال کرتے ہوئے پری ٹرین کیا جاتا ہے۔ پری ٹریننگ کے دوران ماڈل زبان کی سمجھ کا ایک اہم سطح جذب کرتا ہے، جسے پھر دیگر ڈیٹاسیٹس کے ساتھ فائن ٹیوننگ کے ذریعے استعمال کیا جا سکتا ہے۔ اس عمل کو **ٹرانسفر لرننگ** کہا جاتا ہے۔

![تصویر http://jalammar.github.io/illustrated-bert/ سے](../../../../../translated_images/ur/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

ٹرانسفارمر آرکیٹیکچرز کی بہت سی مختلف حالتیں ہیں، جن میں BERT، DistilBERT، BigBird، OpenGPT3 اور مزید شامل ہیں، جنہیں فائن ٹیون کیا جا سکتا ہے۔ [HuggingFace پیکیج](https://github.com/huggingface/) ان آرکیٹیکچرز میں سے بہت سے کو PyTorch کے ساتھ تربیت دینے کے لیے ایک ریپوزٹری فراہم کرتا ہے۔

## متن کی درجہ بندی کے لیے BERT کا استعمال

آئیے دیکھتے ہیں کہ ہم اپنے روایتی کام کو حل کرنے کے لیے پری ٹرین شدہ BERT ماڈل کو کیسے استعمال کر سکتے ہیں: ترتیب کی درجہ بندی۔ ہم اپنے اصل AG News ڈیٹاسیٹ کو درجہ بندی کریں گے۔

سب سے پہلے، آئیے HuggingFace لائبریری اور اپنا ڈیٹاسیٹ لوڈ کریں:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


چونکہ ہم پہلے سے تربیت یافتہ BERT ماڈل استعمال کریں گے، ہمیں ایک خاص ٹوکنائزر کی ضرورت ہوگی۔ سب سے پہلے، ہم ایک ٹوکنائزر لوڈ کریں گے جو پہلے سے تربیت یافتہ BERT ماڈل کے ساتھ منسلک ہے۔

HuggingFace لائبریری میں پہلے سے تربیت یافتہ ماڈلز کا ایک ذخیرہ موجود ہے، جسے آپ صرف ان کے نام `from_pretrained` فنکشنز کے دلائل کے طور پر فراہم کرکے استعمال کر سکتے ہیں۔ ماڈل کے لیے تمام ضروری بائنری فائلیں خودکار طور پر ڈاؤن لوڈ ہو جائیں گی۔

تاہم، بعض اوقات آپ کو اپنے ماڈلز لوڈ کرنے کی ضرورت پڑ سکتی ہے، ایسی صورت میں آپ اس ڈائریکٹری کی وضاحت کر سکتے ہیں جو تمام متعلقہ فائلیں رکھتی ہو، بشمول ٹوکنائزر کے پیرامیٹرز، ماڈل پیرامیٹرز کے ساتھ `config.json` فائل، بائنری ویٹس، وغیرہ۔


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`ٹوکینائزر` آبجیکٹ میں `انکوڈ` فنکشن شامل ہے جو متن کو انکوڈ کرنے کے لیے براہ راست استعمال کیا جا سکتا ہے۔


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

پھر، آئیے ایسے اٹریٹرز بنائیں جنہیں ہم تربیت کے دوران ڈیٹا تک رسائی کے لیے استعمال کریں گے۔ چونکہ BERT اپنی خود کی انکوڈنگ فنکشن استعمال کرتا ہے، ہمیں ایک پیڈنگ فنکشن تعریف کرنے کی ضرورت ہوگی جو `padify` کے مشابہ ہو جو ہم نے پہلے تعریف کیا تھا۔


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

ہماری صورت میں، ہم پہلے سے تربیت یافتہ BERT ماڈل استعمال کریں گے جسے `bert-base-uncased` کہا جاتا ہے۔ آئیے ماڈل کو `BertForSequenceClassfication` پیکج کے ذریعے لوڈ کریں۔ یہ یقینی بناتا ہے کہ ہمارے ماڈل میں پہلے سے ہی درجہ بندی کے لیے مطلوبہ معماری موجود ہے، بشمول آخری درجہ بند کرنے والا۔ آپ کو ایک انتباہی پیغام نظر آئے گا جو بتاتا ہے کہ آخری درجہ بند کرنے والے کے وزن متعین نہیں کیے گئے ہیں، اور ماڈل کو پہلے سے تربیت کی ضرورت ہوگی - یہ بالکل ٹھیک ہے، کیونکہ یہی وہ کام ہے جو ہم کرنے والے ہیں!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

اب ہم تربیت شروع کرنے کے لیے تیار ہیں! چونکہ BERT پہلے سے تربیت یافتہ ہے، ہم ایک چھوٹی سی لرننگ ریٹ سے آغاز کرنا چاہتے ہیں تاکہ ابتدائی وزن خراب نہ ہوں۔

تمام محنت `BertForSequenceClassification` ماڈل کے ذریعے کی جاتی ہے۔ جب ہم تربیتی ڈیٹا پر ماڈل کو کال کرتے ہیں، تو یہ ان پٹ منی بیچ کے لیے نقصان (loss) اور نیٹ ورک آؤٹ پٹ دونوں واپس کرتا ہے۔ ہم نقصان کو پیرامیٹر کی اصلاح کے لیے استعمال کرتے ہیں (`loss.backward()` بیک ورڈ پاس کرتا ہے)، اور `out` کو تربیتی درستگی کا حساب لگانے کے لیے استعمال کرتے ہیں، جہاں حاصل کردہ لیبلز `labs` (جو `argmax` کے ذریعے حساب کیے جاتے ہیں) کو متوقع `labels` کے ساتھ موازنہ کیا جاتا ہے۔

عمل کو کنٹرول کرنے کے لیے، ہم نقصان اور درستگی کو کئی تکراروں کے دوران جمع کرتے ہیں اور ہر `report_freq` تربیتی سائیکل کے بعد انہیں پرنٹ کرتے ہیں۔

یہ تربیت ممکنہ طور پر کافی وقت لے سکتی ہے، اس لیے ہم تکرار کی تعداد کو محدود کرتے ہیں۔


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


آپ دیکھ سکتے ہیں (خاص طور پر اگر آپ iterations کی تعداد بڑھائیں اور کافی دیر انتظار کریں) کہ BERT classification ہمیں کافی اچھی درستگی فراہم کرتا ہے! اس کی وجہ یہ ہے کہ BERT پہلے ہی زبان کی ساخت کو کافی حد تک سمجھتا ہے، اور ہمیں صرف آخری classifier کو fine-tune کرنے کی ضرورت ہوتی ہے۔ تاہم، چونکہ BERT ایک بڑا ماڈل ہے، پورا تربیتی عمل کافی وقت لیتا ہے اور سنجیدہ computational طاقت کی ضرورت ہوتی ہے! (GPU، اور ترجیحاً ایک سے زیادہ)۔

> **نوٹ:** ہمارے مثال میں، ہم نے سب سے چھوٹے pre-trained BERT ماڈلز میں سے ایک استعمال کیا ہے۔ بڑے ماڈلز موجود ہیں جو ممکنہ طور پر بہتر نتائج فراہم کر سکتے ہیں۔


## ماڈل کی کارکردگی کا جائزہ لینا

اب ہم اپنے ماڈل کی کارکردگی کو ٹیسٹ ڈیٹا سیٹ پر جانچ سکتے ہیں۔ جائزہ لینے کا عمل تربیتی عمل سے کافی حد تک مشابہت رکھتا ہے، لیکن ہمیں یہ نہیں بھولنا چاہیے کہ ماڈل کو جائزہ لینے کے موڈ میں تبدیل کرنے کے لیے `model.eval()` کو کال کرنا ضروری ہے۔


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## خلاصہ

اس یونٹ میں، ہم نے دیکھا کہ **transformers** لائبریری سے پہلے سے تربیت یافتہ زبان کے ماڈل کو لینا اور اسے ہمارے متن کی درجہ بندی کے کام کے لیے ڈھالنا کتنا آسان ہے۔ اسی طرح، BERT ماڈلز کو ادارہ نکالنے، سوالات کے جوابات دینے، اور دیگر NLP کاموں کے لیے استعمال کیا جا سکتا ہے۔

Transformer ماڈلز NLP میں جدید ترین معیار کی نمائندگی کرتے ہیں، اور زیادہ تر معاملات میں، یہ وہ پہلا حل ہونا چاہیے جس کے ساتھ آپ اپنی مرضی کے مطابق NLP حل پر تجربہ کرنا شروع کریں۔ تاہم، اگر آپ جدید نیورل ماڈلز بنانا چاہتے ہیں تو اس ماڈیول میں زیر بحث ری کرنٹ نیورل نیٹ ورکس کے بنیادی اصولوں کو سمجھنا انتہائی اہم ہے۔



---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا غیر درستیاں ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ ہم اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے ذمہ دار نہیں ہیں۔
